# Install

In [1]:
!pip install 'git+https://github.com/cene555/ruCLIP-SB.git'

  Cloning https://github.com/cene555/ruCLIP-SB.git to /tmp/pip-req-build-td8cko6v
  Running command git clone -q https://github.com/cene555/ruCLIP-SB.git /tmp/pip-req-build-td8cko6v
     |████████████████████████████████| 431 kB 10.1 MB/s 
     |████████████████████████████████| 3.1 MB 87.9 MB/s 
     |████████████████████████████████| 3.3 MB 81.3 MB/s 
     |████████████████████████████████| 596 kB 59.6 MB/s 
     |████████████████████████████████| 895 kB 55.4 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
  Created wheel for ruclipsb: filename=ruclipsb-1.0-py3-none-any.whl size=5562 sha256=f497e108f0173806c658f6a965ac70ebaf70c702041ded112e66b5958a4b9a19
  Stored in directory: /tmp/pip-ephem-wheel-cache-rmcf_uy5/wheels/6d/97/0f/f70741c9e95bd88c83a32b20d029078aa97fc3e781b4282e9f
Successfully built ruclipsb
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Download model

In [2]:
!gdown https://drive.google.com/uc?id=1-CghuC9TCIDyn5H3zQS6ho_TNiudzJCX 

Downloading...
From: https://drive.google.com/uc?id=1-CghuC9TCIDyn5H3zQS6ho_TNiudzJCX
To: /content/ruCLIP-SB.pkl
100% 159M/159M [00:00<00:00, 282MB/s]


Download dataset

In [ ]:
#download this dataset https://drive.google.com/file/d/1XQYR66ndrMPTy_Z4eD5jMgl_BNISZGfq/view?usp=sharing

In [14]:
!gdown https://drive.google.com/uc?id=1VRDYOvzvASd8PHSBTbp6En47-I5X88YU

Downloading...
From: https://drive.google.com/uc?id=1VRDYOvzvASd8PHSBTbp6En47-I5X88YU
To: /content/ru_flickr.csv
100% 5.73M/5.73M [00:00<00:00, 90.1MB/s]


In [4]:
!unzip /content/flickr.zip

unzip:  cannot find or open /content/flickr.zip, /content/flickr.zip.zip or /content/flickr.zip.ZIP.


# Import

In [5]:
from ruclipsb import ruCLIPSB
from ruclipsb.utils import tokenize, _convert_image_to_rgb
from ruclipsb.trainer import Trainer

import torch
import transformers
from PIL import Image
import transformers
from transformers import BertTokenizer
from torchvision import transforms
import pandas as pd
try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC
import cv2
from sklearn.model_selection import train_test_split

# Load data

In [6]:
df = pd.read_csv('ru_flickr.csv')

In [7]:
df.head()

,image_name,text
0,1000092795.jpg,Два молодых парня с шаткими волосами смотрят н...
1,10002456.jpg,Несколько человек в жестких люках управляют ги...
2,1000268201.jpg,Ребенок в розовом платье поднимается по лестни...
3,1000344755.jpg,Кто-то в синей рубашке и шляпе стоит на полу и...
4,1000366164.jpg,"Двое мужчин, один в темной рубашке, другой в ч..."


In [8]:
train_df, val_df = train_test_split(df, test_size=0.1, random_state=655)

In [9]:
train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)


# Load model

In [10]:
device = 'cuda'

In [11]:
model = ruCLIPSB().to(device)
model.load_state_dict(torch.load('/content/ruCLIP-SB.pkl'))

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth" to /root/.cache/torch/hub/checkpoints/swin_tiny_patch4_window7_224.pth


Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.5M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

# Train model

In [12]:
trainer = Trainer(train_dataframe='/content/train.csv', train_dir='/content/flickr30k_images/flickr30k_images',
                  val_dataframe='/content/val.csv', val_dir='/content/flickr30k_images/flickr30k_images',)

In [ ]:
model = trainer.train_model(model, epochs_num=2, device=device, verbose=20)